In [1]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

np.random.seed(42)
random.seed(42)

# ======================
# 基本設定
# ======================
N = 100

education = ["Bachelor's", "High School", "Master's", "PhD"]
employment = ["Full-time", "Part-time", "Unemployed", "Self-employed"]
marital = ["Married", "Divorced", "Single"]
yes_no = ["Yes", "No"]
loan_purpose = ["Business", "Home", "Education", "Other", "Auto"]

# ======================
# 工具函式
# ======================
def random_loan_id():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

def random_date(start="2013-01-01", end="2018-12-31"):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    return start + timedelta(days=random.randint(0, (end-start).days))

# ======================
# 先生成基礎數值資料
# ======================
df = pd.DataFrame({
    "LoanID": [random_loan_id() for _ in range(N)],
    "Age": np.random.randint(18, 70, N),
    "Income": np.random.randint(15000, 150000, N),
    "LoanAmount": np.random.randint(5000, 250000, N),
    "CreditScore": np.random.randint(300, 850, N),
    "MonthsEmployed": np.random.randint(0, 120, N),
    "NumCreditLines": np.random.randint(1, 5, N),
    "LoanTerm": np.random.choice([12, 24, 36, 48, 60], N),
    "DTIRatio": np.round(np.random.uniform(0.1, 0.9, N), 2),
    "Education": np.random.choice(education, N),
    "EmploymentType": np.random.choice(employment, N),
    "MaritalStatus": np.random.choice(marital, N),
    "HasMortgage": np.random.choice(yes_no, N),
    "HasDependents": np.random.choice(yes_no, N),
    "LoanPurpose": np.random.choice(loan_purpose, N),
    "HasCoSigner": np.random.choice(yes_no, N),
    "Loan Date (DD/MM/YYYY)": [
        random_date().strftime("%Y/%m/%d") for _ in range(N)
    ]
})

# ======================
# 🔥 利率：由 CreditScore + DTI 控制
# ======================
df["InterestRate"] = (
    25
    - (df["CreditScore"] - 300) / 550 * 15      # CreditScore 越高 → 利率越低
    + df["DTIRatio"] * 10                       # DTI 越高 → 利率越高
    + np.random.normal(0, 1.2, N)               # 市場噪音
)

df["InterestRate"] = df["InterestRate"].clip(2, 25).round(2)

# ======================
# 🔥 違約邏輯（風險分數制）
# ======================
risk_score = (
    (700 - df["CreditScore"]) / 400 * 0.4 +     # 信用分數
    df["DTIRatio"] * 0.4 +                      # 負債比
    (df["EmploymentType"] == "Unemployed") * 0.15 +
    (df["Income"] < 40000) * 0.05
)

# 轉成違約機率（0 ~ 1）
default_prob = risk_score.clip(0, 1)

df["Default"] = np.random.binomial(1, default_prob)

df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Loan Date (DD/MM/YYYY),InterestRate,Default
0,XAJI0Y6DPB,56,129752,115078,697,61,3,48,0.25,PhD,Part-time,Married,No,No,Education,Yes,2014/02/11,16.30,0
1,HSAHXTHV3A,69,63984,85623,576,22,4,36,0.47,Bachelor's,Part-time,Single,No,Yes,Home,Yes,2017/02/13,21.38,0
2,3ZMF8MDD4V,46,55774,27671,803,8,1,60,0.38,Master's,Self-employed,Married,Yes,Yes,Business,No,2017/01/09,15.88,0
3,30T9NT3W5U,32,17568,161256,691,11,1,48,0.57,PhD,Full-time,Divorced,Yes,Yes,Education,No,2013/09/06,21.25,0
4,ZBIKCIDKWN,60,135151,98384,434,116,4,60,0.16,Master's,Full-time,Divorced,No,No,Home,No,2017/06/19,21.90,1


In [2]:
df.to_csv(r"C:\Users\User\Desktop\python\Random_Loan.csv", index=False)
pd.read_csv(r"C:\Users\User\Desktop\python\Random_Loan.csv").head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Loan Date (DD/MM/YYYY),InterestRate,Default
0,XAJI0Y6DPB,56,129752,115078,697,61,3,48,0.25,PhD,Part-time,Married,No,No,Education,Yes,2014/02/11,16.30,0
1,HSAHXTHV3A,69,63984,85623,576,22,4,36,0.47,Bachelor's,Part-time,Single,No,Yes,Home,Yes,2017/02/13,21.38,0
2,3ZMF8MDD4V,46,55774,27671,803,8,1,60,0.38,Master's,Self-employed,Married,Yes,Yes,Business,No,2017/01/09,15.88,0
3,30T9NT3W5U,32,17568,161256,691,11,1,48,0.57,PhD,Full-time,Divorced,Yes,Yes,Education,No,2013/09/06,21.25,0
4,ZBIKCIDKWN,60,135151,98384,434,116,4,60,0.16,Master's,Full-time,Divorced,No,No,Home,No,2017/06/19,21.90,1
